# Depth maps

In [1]:
########################################################################################
# Author: Justin Clancy, University of Melbourne, 2020                                 #
# Creating depth maps for stereo images                                                #
########################################################################################

### Import modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve
from PIL import Image
import cv2 as cv
import os

### Import images from a folder

In [3]:
# Define a function to import all images in a folder

def import_images(folder):
    # create an empty list to fill with image data
    images = []
    # Search through the folder for files
    for filename in os.listdir(folder):
        # An image is loaded
        image = cv.imread(os.path.join(folder,filename))
        # Add the image to the list if it actually has
        # data in it
        if image is not None:
            images.append(image)
    # Since all our folders just have 2 images
    # save the first as the pattern (left) and second
    # as the template (right)
    pattern_col = images[0]
    template_col = images[1]
    # Convert to greyscale with rgb weights
    cols = [0.2989,0.5870,0.1140]
    pattern = np.dot(pattern_col[...,:3],cols)
    template = np.dot(template_col[...,:3],cols)
    return pattern, template

### Get windows

#### No overlap:

In [4]:
# Define a window function to take the left image and 
# cut it into n sub windows that preserve the 'physical'
# layout of the input

def windows(left_im, y_win, x_win):
    # Obtain the dimensions of the left image
    height, width = left_im.shape
    # Condition that the input num x and y windows must be
    # divisible by the height and width of the original image
    assert height % y_win == 0,"{} width isn't evenly divisble by {}!".format(height, y_win)
    assert width % x_win == 0,"{} height isn't evenly divisble by {}".format(width, x_win)
    # Reshape the input image by the desired x and y input
    return (left_im.reshape(height//y_win, y_win, -1, x_win)
               .swapaxes(1,2)
               .reshape(-1, x_win, y_win))

#### With overlap:

In [5]:
# A function to follow the above idea, howver allowing
# for the windows to overlap by a predefined
# amount.

def overlap_windows(left_im, y_win, x_win, overlap):
    
    # Define array size length in bytes and dimensions in x and y
    size = left_im.itemsize
    shape = left_im.shape

    # Strides map indicies of one array to positions in the data
    # buffer, this process here is mapping out the windows to slide
    # across the left image
    strides = (size*shape[1]*(y_win-overlap), 
               size*(x_win-overlap),
               size*shape[1], size)
    # Window dimensions
    shape = (int((shape[0] - x_win)/(x_win-overlap))+1,
             int((shape[1] - y_win)/(y_win-overlap))+1 ,
             y_win, x_win)
    windowsy=shape[0]
    windowsx=shape[1]

    # Return a view of the input array with the defined shape and strides
    # i.e., return a cut up version of the left image
    return np.lib.stride_tricks.as_strided(left_im, strides=strides,
                                              shape=shape ).reshape(-1, y_win,
                                                                    x_win),windowsy,windowsx

### Define the search area

In [6]:
# The next step is to define the search area
# For each window in the left we need to look for its location in the right

def find_window(right_im,center,win_dim,im_dim,num_win):

    # Get center coordinates from the center of a window
    center_x,center_y = center
    
    # Maximum value of x and y from template original images
    maxy,maxx = im_dim 
    
    # Get left image window dimensions
    winx,winy = win_dim
    # Define the search area as multiples of the windows in a square
    search_x = winx*num_win
    search_y = winy*num_win
    
    # Define the area in the right image to search
    # minimum x coordinate, making sure its cut off at zero
    left = center_x - search_x
    if left <= 0:
        left = 0
        right = left + search_x
    else:
        left = left
        right = left + search_x
        if right >= maxy:
            right = maxy
            left = maxy - search_x
        else: 
            left = left
            right = right

    # minimum y coordinate ensuring it cuts off at zero
    bottom = center_y - search_y
    if bottom <= 0:
        bottom = 0
        top = bottom + search_y
    else:
        bottom = bottom
        top = bottom + search_y
        if top >= maxx:
            top = maxx
            bottom = maxx - search_y
        else:
            top = top
            bottom = bottom
        
    # Return the search window in the right image
    
    right_win = right_im[left:right,bottom:top]

    return right_win

### Window offset

In [7]:
# Now offset for a window.
# This function should take the pattern window and a search area, 
# returning the best lag (x,y) for the window in the search area.

def b_lag(scores,pattern):
    # Check for the best lag as a position in x and y separetly
    y,x  = np.unravel_index(np.argmax(scores,axis=None),scores.shape)
    # Make sure the best lag points correspond to roughly the centre of the
    # pattern so if we put a marker on this location its right in the middle
    xmid = x + pattern.shape[1]/2
    ymid = y + pattern.shape[0]/2
    high_score = np.max(scores)
    return xmid,ymid,high_score

def window_offset(pattern_window,search_area):
    # Mean shift the windowed pattern and template
    #mean_s = search_area - np.mean(search_area)
    #mean_p = pattern_window - np.mean(pattern_window)
    mean_s = pattern_window
    mean_p = pattern_window
    empty_template = np.ones(mean_s.shape)
    score_array = fftconvolve(mean_p,mean_s.conj(),mode='full')
    
    mean_p = fftconvolve(np.square(mean_p),empty_template,mode='full') - \
                np.square(fftconvolve(mean_p,empty_template,mode='full')) \
                    / (np.prod(mean_s.shape))
    
    mean_p[np.where(mean_p < 0)] = 0
    mean_s = np.sum(np.square(mean_s))
    score_array = score_array / np.sqrt(mean_p * mean_s)
    
    score_array[np.where(np.logical_not(np.isfinite(score_array)))]
    bl_y,bl_x,s_max = b_lag(score_array,pattern_window)
    
    return bl_x,bl_y,s_max,score_array
    
from skimage.feature import match_template
def test(pattern,template):
    score_arr = match_template(pattern,template)
    bl_y,bl_x,s_max = b_lag(score_arr,pattern)
    return s_max

## Depth Map

In [8]:
def depth_map(folder, window_size = (20,20), overlap_perc = 50, search_ext = 3):
    # Import the left and right image from folder
    pattern,template = import_images(folder)
    
    # Define image dimensions
    templatey,templatex = template.shape
    patterny,patternx = pattern.shape
    
    # The images have slightly different shapes so make them equal 
    # with slight padding
    new_dimx = patternx + 2
    new_dimy = patterny + 2
    right_im = np.zeros((new_dimy,new_dimx))
    right_im[:templatey,:templatex] = template
    left_im = np.zeros((new_dimy,new_dimx))
    left_im[:patterny,:patternx] = pattern
    
    # Define overlap from input percentage
    win_x = window_size[1]
    win_x0 = np.int(win_x/2)   # Center of window (x)
    win_y = window_size[0]
    win_y0 = np.int(win_y/2)   # Center of window (y)
    olap = np.int(win_x/(100/overlap_perc))
    
    # Cut up the left image into equal sized windowed of defined
    # shape and overlap 
    left_windows = overlap_windows(left_im,win_y,win_x,olap)

    window_dim = (left_windows[1],left_windows[2])
    
    #olap = win_x - olap
    depth_map = np.zeros(window_dim)
    winy = left_windows[1]
    winx = left_windows[2]

    
    for i in np.arange(winy):
        for j in np.arange(winx):
            search = find_window(template,(win_y0 + i*olap,win_x0 + j*olap),(win_y,win_x),pattern.shape,search_ext)
            score = test(search,left_windows[0][i*winx+j])
            depth_map[i,j] = depth_map[i,j] + score

    plt.imshow(depth_map)

    plt.title('Depth Map')
    cbar = plt.colorbar(orientation='horizontal')
    cbar.set_label('Cross-Correlation Score')
    plt.show()
    
    # Some of the clearest results can be found with the following settings
    
    # Block:
    # Window size = (25,25)
    # Search extent = 10
    
    # Rubiks Cube:
    # Window size = (15,15)
    # Search extent = 5
    
    # Giraffes: 
    # Window size = (25,25)
    # Search extent = 20
    
    # Pinecone:
    # Window size = (50,50)
    # Search extent - 4
    

# Run the function 

In [1]:
# Input folder destination, window size dimensions, overlap percentage and search extent multiplier

depth_map('',window_size = (,), overlap_perc=,search_ext=)